In [1]:
from src.helper_functions import output_repeated_block, find_block
from src.symmetric_encryption import is_ecb_mode, determine_blocksize
from src.xor import xor_bytes
import src.load as load
import src.aes128 as aes128
import src.convert as convert
import src.padding as padding
# 
import base64
import secrets

## **Challenge 9: Implement PKCS#7 padding**

In [2]:
padding.apply_pkcs_7(b"YELLOW SUBMARINE", 20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

## **Challenge 10: Implement CBC mode**

In [3]:
key = secrets.token_bytes(16)
iv = secrets.token_bytes(16)
text = b'YELLOW SUBMARINE'

cyphertext = aes128.cbc_encrypt(text, key, iv)
plaintext = aes128.cbc_decrypt(cyphertext, key, iv)

print(cyphertext)
print(plaintext)

b'\xdaG\x86\x920%y-\x9b\x91$vN\xcax\xe3'
b'YELLOW SUBMARINE'


In [4]:
file_bytes = load.file_as_b64("challenge_data/10.txt", remove_newlines=True)

key = b'YELLOW SUBMARINE'
IV = b'\x00' * 16
aes128.cbc_decrypt(file_bytes, key, IV).decode('utf-8')

"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take a

## **Challenge 11: An ECB/CBC detection oracle**

In [5]:
def encryption_oracle_11(plaintext: bytes) -> bytes:
    """Encrypts a `plaintext` with AES-128 in either ECB or CBC mode. The plaintext is padded with PKCS#7 padding
    and a random prefix and suffix are added to the plaintext. The key and IV are chosen at random. The function
    returns a tuple of the mode used and the cyphertext. 
    
    `encrypt(before_bytes + plaintext + after_bytes)`
    """

    key = secrets.token_bytes(16)  # for AES-128
    iv = secrets.token_bytes(16)  # for AES-128

    before_bytes = secrets.token_bytes(secrets.choice([5, 6, 7, 8, 9, 10]))
    after_bytes = secrets.token_bytes(secrets.choice([5, 6, 7, 8, 9, 10]))

    modified_plaintext = padding.apply_pkcs_7(before_bytes + plaintext + after_bytes, 16)

    if secrets.choice([True, False]):
        print("ECB")
        return aes128.ecb_encrypt(modified_plaintext, key)
    else:
        print("CBC")
        return aes128.cbc_encrypt(modified_plaintext, key, iv)

In [6]:
for _ in range(5):
    ecb_mode = is_ecb_mode(encryption_oracle_11, 16)
    if ecb_mode:
        print("ECB", "\n")
    else:
        print("CBC", "\n")

ECB
ECB 

ECB
ECB 

CBC
CBC 

CBC
CBC 

ECB
ECB 



## **Challenge 12: Byte-at-a-time ECB decryption (Simple)**

In [7]:
# Assume that key is unknown but same for all encryptions
challenge_12_key = secrets.token_bytes(16)

In [8]:
def encryption_oracle_12(plaintext: bytes) -> bytes:
    """Encrypts a `plaintext` with AES-128 in ECB mode. The plaintext is padded with PKCS#7 padding.
    There is an unknown string appended to the plaintext. The key is chosen at random. The function
    returns the cyphertext."""
    unknown_string = base64.decodebytes(b"Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK")
    padded_plaintext = padding.apply_pkcs_7(plaintext + unknown_string, 16)
    return aes128.ecb_encrypt(padded_plaintext, challenge_12_key)

In [9]:
blocksize, offset = determine_blocksize(encryption_oracle_12)
print(f"Block length: {blocksize}")
print(f"Is this encrypted with ECB mode: {is_ecb_mode(encryption_oracle_12, blocksize)}")

# make dictionary of all possible last bytes by brute forcing the block by block
plaintext = b''

def gen_lookup_table(plaintext: bytes, blocksize: int, oracle: callable) -> dict:
    """Generates a lookup table for the last byte of the plaintext.
    The lookup table maps the cyphertext to the last byte of the plaintext."""
    cypher2plain = {}                                   # maps cyphertext to plaintext
    w = plaintext[(1-blocksize):]                      # window of last blocksize - 1 bytes of plaintext
    pad = b'A' * (blocksize - len(w) - 1)              # used to pad the first block *ONLY*
    for i in range(256):                                # iterate through all possible final bytes
        p = pad + w + convert.int2byte(i)                      # generate plaintext to encrypt
        cy = oracle(p)                                  # encrypt the plaintext
        cypher2plain[cy[:blocksize]] = convert.int2byte(i)    # add to dictionary
    return cypher2plain


def padded_encryption(plaintext: bytes, blocksize: int, oracle: callable) -> bytes:
    """Pad and encrypt the plaintext. The padding is done so that a full block only has a single unknown byte."""
    pad = b'A' * ((blocksize - len(plaintext) - 1) % blocksize)
    return oracle(pad)


for char_i in range(10000): # 10000 is an arbitrary large number
    cyphertext = padded_encryption(plaintext, blocksize, encryption_oracle_12)

    # When the full plaintext is found, the last block will not be in the dictionary
    block_num = char_i // blocksize    # current block number
    selected_cypher = cyphertext[block_num*blocksize:(block_num + 1)*blocksize]
    try:
        cypher2plain = gen_lookup_table(plaintext, blocksize, encryption_oracle_12)
        plaintext += cypher2plain[selected_cypher]
    except KeyError: 
        break


plaintext.decode('utf-8')

Block length: 16
Is this encrypted with ECB mode: True


"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x01"

## **Challenge 13: ECB cut-and-paste**

In [10]:
challenge_13_key = secrets.token_bytes(16)

In [11]:
def profile_for(email: str):
    """Returns a profile for an email address. The profile is a string of the form:
    `email=<email>&uid=<10 random digits>&role=user`
    """
    email = email.replace("&", "").replace("=", "") # eat the characters (yum yum!)
    uid = 1000 + secrets.randbelow(10**4 - 1000)    # 1000 <= uid < 10**4
    object = f"email={email}&uid={uid}&role=user"   # the object to be encrypted
    padded_object = padding.apply_pkcs_7(object.encode('utf-8'), 16)    # pad the object
    return aes128.ecb_encrypt(padded_object, challenge_13_key)          # encrypt the object

def decrypt_profile(profile: bytes) -> dict:
    """Decrypts a profile and returns a dictionary of the form:
    `{'email': <email>, 'uid': <uid>, 'role': <role>}`
    """
    decrypted_profile = aes128.ecb_decrypt(profile, challenge_13_key)
    decrypted_profile = padding.remove_pkcs_7(decrypted_profile)
    decrypted_profile = decrypted_profile.decode('utf-8')
    return convert.string2dict(decrypted_profile)

In [12]:
blocksize = 16

# 10 A's push the admin string into its own block
a = 'A' * 10

# admin padding with correct padding bytes
admin_str = padding.apply_pkcs_7(b'admin', 16).decode('utf-8')

# get the encrypted profile for the admin string
admin_encrypted_profile = profile_for(a + admin_str)

# extract the encrypted admin string
admin_bytes = admin_encrypted_profile[blocksize:2*blocksize]

# 1. get the encrypted profile for the user
# 2. remove the last block
# 3. append the admin bytes
hacker_encrypted_profile = profile_for("hackerman69")
hacker_encrypted_profile = hacker_encrypted_profile[:-blocksize]
hacker_encrypted_profile += admin_bytes

print("Successfully created an encrypted profile for the admin user!")
print(decrypt_profile(hacker_encrypted_profile))

Successfully created an encrypted profile for the admin user!
{'email': 'hackerman69', 'uid': '7513', 'role': 'admin'}


## **Challenge 14: Byte-at-a-time ECB decryption (Harder)**

In [13]:
encryption_oracle_14_key = secrets.token_bytes(16)

In [14]:
def encryption_oracle_14(plaintext: bytes) -> bytes:
    """Encrypts a `plaintext` with AES-128 in ECB mode. The plaintext is padded with PKCS#7 padding.
    `AES-128-ECB(random-prefix || attacker-controlled || target-bytes, random-key)`
    """
    max_num_bytes = 32
    rand_bytes = secrets.token_bytes(secrets.randbelow(max_num_bytes))
    unknown_string = base64.decodebytes(b"Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK")
    return aes128.ecb_encrypt(padding.apply_pkcs_7(rand_bytes + plaintext + unknown_string, 16), encryption_oracle_14_key)

In [15]:
blocksize, offset = determine_blocksize(encryption_oracle_14)
print(f"Block length: {blocksize}")
print(f"Padding offset (bytes to add so that there is no padding): {offset}")
print(f"Is this encrypted with ECB mode: {is_ecb_mode(encryption_oracle_14, blocksize)}")

def altered_oracle(plaintext: bytes, blocksize: int) -> bytes:
    """The oracle with the altered plaintext"""
    
    # use flag to determine when the inserted plaintext starts
    flag = b'F' * blocksize
    cy_flag = output_repeated_block(encryption_oracle_14(3*flag), blocksize)
    while True:
        cy = encryption_oracle_14(flag + plaintext)
        i = find_block(cy, cy_flag, blocksize)
        if i:
            return cy[i+blocksize:]

# alter the oracle so that the inserted plaintext starts at the beginning of a block
oracle_14_no_rand_bytes = lambda x: altered_oracle(x, blocksize)

plaintext = b''
for char_i in range(10000): # 10000 is an arbitrary large number
    cyphertext = padded_encryption(plaintext, blocksize, oracle_14_no_rand_bytes)

    block_num = char_i // blocksize    # current block number
    selected_cypher = cyphertext[block_num*blocksize:(block_num + 1)*blocksize]
    try:
        cypher2plain = gen_lookup_table(plaintext, blocksize, oracle_14_no_rand_bytes)
        plaintext += cypher2plain[selected_cypher]
    except KeyError: 
        break

print(plaintext)

Block length: 16
Padding offset (bytes to add so that there is no padding): 0
Is this encrypted with ECB mode: True
b'R\xd4\xf5'


## **Challenge 15: PKCS#7 padding validation**

In [16]:
padding.remove_pkcs_7(b"ICE ICE BABY\x04\x04\x04\x04")

b'ICE ICE BABY'

In [17]:
try:
    padding.remove_pkcs_7(b"ICE ICE BABY\x05\x05\x05\x05")
except ValueError:
    print("Successfully raised a ValueError!")

Successfully raised a ValueError!


In [18]:
try:
    padding.remove_pkcs_7(b"ICE ICE BABY\x01\x02\x03\x04")
except ValueError:
    print("Successfully raised a ValueError!")

Successfully raised a ValueError!


## **Challenge 16: CBC bitflipping attacks**

In [19]:
challenge_16_key = secrets.token_bytes(16)
challenge_16_iv = secrets.token_bytes(16)

In [20]:
def sandwich(userdata: str) -> str:
    """Sandwich `userdata` string in between `comment1` and `comment2`.

    `comment1 || userdata || comment2`
    """
    prepend = "comment1=cooking%20MCs;userdata="
    append = ";comment2=%20like%20a%20pound%20of%20bacon"
    # The function quotes out the ";" and "=" characters
    # This is designed to prevent the user from injecting their own
    # "admin=true" string.
    userdata = userdata.replace(";", "%3B").replace("=", "%3D")
    return prepend + userdata + append


def encryption_oracle_16(userdata: str) -> bytes:
    """Encrypts `userdata` with AES-128 in CBC mode."""
    # we must apply the sandwich function so that the user cannot inject
    # their own `admin=true` string
    plaintext = convert.string2bytes(sandwich(userdata))
    return aes128.cbc_encrypt(
        padding.apply_pkcs_7(plaintext, 16), 
        challenge_16_key, 
        challenge_16_iv,
    )

def is_admin_oracle(cyphertext: bytes) -> bool:
    """Checks if the decrypted cyphertext contains the string `;admin=true;`"""
    
    # decrypt the cyphertext
    plaintext = padding.remove_pkcs_7(
        aes128.cbc_decrypt(
            cyphertext,
            challenge_16_key, 
            challenge_16_iv,
        )
    )
    
    # convert plaintext to a dictionary
    plaintext_dict = convert.string2dict(
        plaintext.decode('utf-8', 'ignore'), 
        equal_sign="=", 
        break_sign=";",
    )

    print("is_admin_oracle received:", plaintext_dict)

    # check if the dictionary contains the key `admin` and if it's value is `true`
    try:
        return plaintext_dict['admin'] == 'true'
    except KeyError:
        return False

convert.string2dict(
    sandwich("hello;world=foo"), 
    equal_sign="=", 
    break_sign=";",
)

{'comment1': 'cooking%20MCs',
 'userdata': 'hello%3Bworld%3Dfoo',
 'comment2': '%20like%20a%20pound%20of%20bacon'}

In [21]:
bytes_encryption_oracle_16 = lambda b: encryption_oracle_16(b.decode('utf-8'))
blocksize, offset = determine_blocksize(bytes_encryption_oracle_16)
print(f"Block length: {blocksize}")
print(f"Padding offset (bytes to add so that there is no padding): {offset}")
print(f"Is this encrypted with ECB mode: {is_ecb_mode(bytes_encryption_oracle_16, blocksize)}\n")

# insert the attacker controlled string into the cyphertext
a_str = "a" * 16
original_cyphertext = encryption_oracle_16(a_str)

# find the block that contains the attacker controlled string
# for this example we know that it is the first block
ith_block = 1

# this is the string that will be inserted into the cyphertext
# a=aa is just a filler to make the string 16 bytes long
admin_str = b";admin=true;a=aa" 

# xor the old block with the attacker controlled string as well as the input block
# this converts the plaintext in the attacker controlled block to the admin_str
input_block = convert.string2bytes(a_str)
old_block = cyphertext[blocksize*ith_block:blocksize*(ith_block+1)]
new_block = xor_bytes(old_block, input_block)
new_block = xor_bytes(new_block, admin_str)

# replace the old block with the new block
cyphertext = cyphertext[:blocksize*ith_block] + new_block + cyphertext[blocksize*(ith_block+1):]

# check if the cyphertext contains the string `;admin=true;`
is_admin = is_admin_oracle(cyphertext)
print(f"\nWe have successfully injected the string ';admin=true;' into the cyphertext!")
print(f"admin: {is_admin}")

Block length: 16
Padding offset (bytes to add so that there is no padding): 5
Is this encrypted with ECB mode: False



AssertionError: The byte string must be a multiple of the blocksize